# Joint inversion of source location and subsurface structure from body waves (P,S) and Rayleigh Waves (LR) arrival times
Froment, M., Brissaud, Q., Näsholm, S. P. and Schweitzer, J. (2025) _Balloon seismology enables subsurface inversion without ground stations_

This notebook presents the different steps necessary to run the inversion of source and subsurface for the December 14, 2021 $M_w$ 7.3 earthquake in the Flores Sea. The example inversion uses pressure waveforms recorded by four balloons of the Strateole2 campaign flying over Oceania during the event. First, the balloon data is processed to reduced the influence of low-frequency buoyancy oscillations. Then, P, S and LR phases arrival times are extracted manually from the processed pressure waveforms and formatted for the inversion. A short test inversion ($10^3$ steps) is run. The Monte Carlo inversion results are then further processed and relevant figures shown in the article are produced.     

In [19]:
import MCMC_modules as mc
import numpy as np
import sys
import importlib 
# %matplotlib widget
### Necessary to use interactive picking figures 
%matplotlib qt 

# Balloon Data processing
Processing is done through the `prepare_data_flores_Strateole.py` code. The raw balloon data and the processed outputs are saved in `.mseed` format is in the `./Flores_data/` directory.

In [20]:
import prepare_data_flores_Strateole as bpr
importlib.reload(bpr)
data_source = "./Flores_data/"

### The processed miniseed files have already been calculated. 
### Change to False to redo the processing
skip =True
if not skip:
    ### STEP 1: INITIALIZE STREAMS 
    bdata = bpr.BALLOON_DATA()
    
    ### STEP 2: CORRECT GAPS 
    list_streams = bpr.correct_gaps(bdata)
    
    ### STEP 3: UPSAMPLE Z 
    list_streams_aligned = bpr.Z_upsample_align(list_streams, do_plot=False)
    
    ### STEP 4: Determine relation between P and Z 
    list_streams_opt = bpr.P_Z_relation(list_streams_aligned, do_plot=False)
    
    ### STEP 5: Obtained corrected pressure, plot, save 
    list_streams_corr = bpr.P_corrected(list_streams_opt, do_plot=False)
    
    ### STEP 6: Save raw and corrected seeds for inversion 
    bpr.save_streams_inversion(data_source, list_streams_corr)

# Phase picking and formatting
In the following step, we will prepare the inversion setup and proceed to pick the arrivals. This step is very important, as it will define the name of our run and where the inputs and outputs of the run will be stored. This step generate a class `DATA`containing the necessary arrays. 

In [21]:
import initialise_inversion_flores_Strateole as data_init
importlib.reload(data_init)
importlib.reload(mc)

data_dir = "./chains_emcee_flores_Strateole/DATA/"
### The picks have already been calculated
### Set do_plot_ftan to True to plot all the FTAN analysis or redo it
DATA = data_init.generate_data(data_dir, initialize=True, do_plot_ftan=False)

Building data to invert
Loading recorded data.
Computing FTAN.
Start phase picker... 
Loading recorded data.
Computing FTAN.
Start phase picker... 
Loading recorded data.
Computing FTAN.
Start phase picker... 
Loading recorded data.
Computing FTAN.
Start phase picker... 
Saving MCMC Files


# Preparing the inversion
Now that `DATA` is prepared, we can set up the inversion. This step generate a class `MCMC` that contains functions to initialise, load and run the McMC samplers, and also contains the results and basic functions to post-process them.

### Step 1: Defined the prior
Prior bounds are defined in a separate text file, here named `prior_flores.txt`. Some precautions: 
- The variable names in the first column should not be changed: the first four variables are relative to the source, then the subsurface velocity parameters follow.
- This files defines the number of layers that will be inverted. There should always be one more line for `vs` and `poisson` than for `h_layer`. If there are 6 lines for `h_layer`, then it means a model with 6 layers and 1 halfspace will be inverted.
- The two columns `prior_min` and `prior_max` define uniform prior bounds for each variable. The two columns `start_min` and `start_max` define restricted bounds, which help initialise the Monte Carlo inversion in a parameter region that is not too pathological. The starting bounds must always be comprised withing the prior bounds, or an error will be raised.

### Step 2: Define variables for inversion

In [22]:
### Name of the run: must be the same as in "data_dir"
name_run      = "flores_Strateole"

### Directory to save results. Must be the same as in "data_dir"
save_dir      = data_dir.split("DATA/")[0]
data_dir      = data_dir

### File containing the priors
param_file    = "prior_flores.txt"

### McMC sampler. emcee is the most optimized option here.
method        = "emcee"

### Number of desired iterations
n_iter        = int(1e2)
### Long, converged inversion (takes a day)
#n_iter        = int(1e6)

### Number of CPUs used. n_cpus=1 preferred for debugging. 
n_cpus        = int(4)   ### home computer
# n_cpus        = int(20)   ### cluster

### To display progress bar (must be set to false if running through a cluster) 
progress      = True

### If set to True, generate a brand new inversion. 
### If set to False, restarts an inversion from the last steps stored in "save_dir"
reset_backend = True

importlib.reload(mc)
MCMC = mc.MCMC_Model(DATA, name_run, save_dir, data_dir, param_file, 
                    method=method, n_iter=n_iter, n_cpus=n_cpus, 
                    progress=progress, reset_backend=reset_backend)

20 CPUs available
4 CPUs running
doing emcee...
Deleting previous backend...


# Run the McMC sampler

Diagnostic information about run (timing, CPUs, crash, reason for crash) are stored in the file `timing_[method].txt`. 

In [23]:
### Will do a warm start if reset_backend=False = continue from the end of the last run. 
### Else it erases everything ans starts over. 
MCMC.run()

Reset start
  2%|▊                                          | 2/100 [00:00<00:06, 15.33it/s]

/nobackup/marouchka/miniconda/envs/env_mcmc/lib/python3.13/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]


100%|█████████████████████████████████████████| 100/100 [00:10<00:00,  9.51it/s]
- SUCCESS -


# Visualize results

The `MCMC` class possesses simple plotting functions to analyse inversion results. 
### General inspection
The first function is called `inspect_chain()` and allows to inspect the behavior of the chain during the inversion. It will display: 
- The evolution of the log_likelihood during the inversion. This plot can be used to identify "stuck" chains and the `crit` parameter (see below)
- The evolution of the acceptance rate
- The evolution of each inverted parameter (if using `emcee`, there are multiple curves for each walker and the least-misfit walker is plotted in red), the histogram of the first parameter and an estimate of the autocorrelation time of the chain for this parameter.

The `thin` parameter controls the amount of chain points that should be discarded. It is here set to display only 1000 chain iterations.

In [24]:
importlib.reload(mc)
MCMC._load_results()

### Comment to ignore
MCMC.inspect_chain(param = 0, direction = "horizontal", thin=max(1,int(n_iter//1000)), do_save=False)

Loading data...
Calculating autocorrelation time...
100%|█████████████████████████████████████████| 49/49 [00:00<00:00, 1613.28it/s]
Displaying misfit...
100%|█████████████████████████████████████████| 49/49 [00:00<00:00, 5356.57it/s]
Displaying acceptance rate...
100%|█████████████████████████████████████████| 49/49 [00:00<00:00, 6759.67it/s]
Displaying chains...
100%|███████████████████████████████████████████| 49/49 [00:02<00:00, 18.19it/s]


### Autocorrelation and convergence

This function calculates the autocorrelation time $\tau$ of chains to evaluate their convergence, following https://emcee.readthedocs.io/en/stable/tutorials/autocorr/. A chain is considered converged when its autocorrelation time becomes stable. `param` is the parameter index from whose chain the autocorrelation will be estimated. `discard` allows to throw away the burn-in iterations before calculating the autocorrelation time.

In [15]:
MCMC.visualize_convergence(discard=1, N_point_autocorr = 50, param = 0, do_save=True)

Loading data...
Calculating autocorrelation times...
100%|██████████████████████████████████████████| 49/49 [00:00<00:00, 985.03it/s]


### Source and subsurface solutions 

The following function plot some diagnostic information on the inverted source location and time, and the posterior subsurface models. Before being run, they require the solutions to be processed: some _burn-in_ iterations will be discarded (option `discard`) and only a selection of iterations will be plotted (option `thin`).

Chains that could be "stuck" in a region of very low likelihood can be discarded as well using a threshold on the log-likelihood with the option `crit`. or the balloon inversion, after a well converged inversions, it should be `crit=-135`.

A estimate of the Maximum A Posteriori (MAP) values for the model will be estimated. Note that it requires a large number of iterations to be trustable. 

In [25]:
importlib.reload(mc)

### Set the minimum log-likelihood to acount for in plots, values with higher misfit (lower log-likelihood) will be discarded. 
crit = -9999
#crit = -135

MCMC.process_solutions(discard=0, thin=1, crit=crit)
### Uncomment for inversions with a very large number of iterations:
#MCMC.process_solutions(discard=1000, thin=2000, crit=crit)

### SHALLOW Subsurface
dep_shallow = np.linspace(0., 100., 201)
MCMC.visualize_solution(figsize=(10,5), Nmod=200, bins_hist=20, bins_wave=50, hspace=0.1, wspace=0.15,  
                            depth_interpolation=dep_shallow, preserve_aspect=True, do_save=True, do_truth=True)
### DEEPER subsurface
dep_deep = np.linspace(0., 1000., 201)
MCMC.visualize_solution(figsize=(10,5), Nmod=200, bins_hist=20, bins_wave=50, hspace=0.1, wspace=0.15,  
                            depth_interpolation=dep_deep, preserve_aspect=True, do_save=True, do_truth=True)

### Source location 
MCMC.source_location(figsize=(4,4), do_save=True, geography = True, zoom=True)

Loading samples...
Extracted 4508 subsurface models
Interpolating models...


100%|██████████████████████████████████| 4508/4508 [00:00<00:00, 5382272.26it/s]


Finding MAP using 4508 iterations
estimated bandwidth:  0.779312977649295
fit meanshift
Mean Shift Estimated Mode: [-7.77446832e+00 -8.06624901e+00  1.22744864e+02  1.05932821e+02
  2.67728471e+00  3.81981828e+00  4.22618424e+00  4.56276924e+00
  4.80981626e+00  5.46862385e+00  5.70464283e+00  2.03233125e-01
  2.25383515e-01  2.69484789e-01  2.69402604e-01  2.61045450e-01
  2.30942270e-01  2.40115563e-01  2.81992481e+00  1.55581933e+01
  1.33216979e+01  1.74809297e+01  2.08017565e+02  2.21014038e+02]
MAP model: [-7.774, -8.066, 122.7, 105.9, 2.677, 3.82, 4.226, 4.563, 4.81, 5.469, 5.705, 0.2032, 0.2254, 0.2695, 0.2694, 0.261, 0.2309, 0.2401, 2.82, 15.56, 13.32, 17.48, 208, 221]
Likelihood of best:  -128.0895164162786
Calculating group velocity and arrival times...
100%|████████████████████████████████████████| 201/201 [00:01<00:00, 161.24it/s]
Interpolating models...


100%|██████████████████████████████████| 4508/4508 [00:00<00:00, 5736626.95it/s]


Likelihood of best:  -128.0895164162786
Calculating group velocity and arrival times...
100%|████████████████████████████████████████| 201/201 [00:01<00:00, 161.62it/s]


### Marginal distributions of parameters
The following two function make corner plots of all or a selection of couples of parameters. This allows to visualise trade-off between two dimensions. 

In [17]:
importlib.reload(mc)

### VERY BIG CORNER PLOT 
### Comment to ignore 
MCMC.corner_plot(do_save=True, do_MAP = True)

### Marginals for a selection of parameters 
### Comment to ignore 
MCMC.parameter_marginals(pars=[[0,1],[0,2],[1,8],[2,8],[5,8],[4,14],[0,8]], find_limits=True, 
                        do_save=True, do_MAP=True)

100%|███████████████████████████████████████████| 24/24 [00:01<00:00, 20.11it/s]
Limits = [[-25.9, 24.3], [-13.1, 5.6], [103.1, 130.4], [1.0, 200.0], [0.5, 4.0], [2.7, 5.6], [3.2, 5.7], [2.0, 6.0], [4.2, 6.0], [4.0, 7.0], [5.4, 6.9], [0.1, 0.4], [0.1, 0.4], [0.1, 0.4], [0.1, 0.4], [0.1, 0.4], [0.1, 0.4], [0.2, 0.3], [0.2, 5.0], [1.0, 30.0], [0.9, 26.9], [-0.8, 32.8], [100.0, 400.0], [100.0, 400.0]]
Limits = [[-25.9, 24.3], [-13.1, 5.6]]
Limits = [[-25.9, 24.3], [103.1, 130.4]]
Limits = [[-13.1, 5.6], [4.2, 6.0]]
Limits = [[103.1, 130.4], [4.2, 6.0]]
Limits = [[2.7, 5.6], [4.2, 6.0]]
Limits = [[0.5, 4.0], [0.1, 0.4]]
Limits = [[-25.9, 24.3], [4.2, 6.0]]


()

### Additional information: the Priors
The prior distribution of subsurface and source parameters can be plotted with the following function.

In [18]:
importlib.reload(mc)

### Comment to ignore 
MCMC.visualize_priors(figsize=(10,5), Nmod=2000, bins_hist=20, bins_wave=50, hspace=0.1, wspace=0.15, 
                           depth_interpolation=dep_shallow, do_save=False, preserve_aspect=True)

Interpolating prior models...
100%|████████████████████████████████████████| 83/83 [00:00<00:00, 21531.87it/s]


ValueError: Cannot take a larger sample than population when 'replace=False'